<a href="https://colab.research.google.com/github/lucimarNeves/analise_dados_dio/blob/main/analiseSentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy as tw
import pandas as pd
import numpy as np
import re

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from textblob import TextBlob

In [ ]:
from tweepy import Cursor

In [ ]:
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""
CONSUMER_KEY = ""
CONSUMER_SECRET = ""

In [ ]:
auth = tw.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
if(not api):
    print("Authenticcation failed!")
    sys.exit(-1)

In [ ]:
BRAZIL_WOE_ID = 23424768
woeid = BRAZIL_WOE_ID

In [ ]:
trends = api.trends_place(id = woeid, exclude = "hashtags")

In [ ]:
searchString = ['alcool', 'gasolina']

In [ ]:
tweets = tw.Cursor(api.search, q = searchString,lang='pt', since='2021-09-24', tweet_mode='extended').items(100)
all_tweets = [tweet.full_text for tweet in tweets]

In [ ]:
print(all_tweets)

In [ ]:
dados = pd.DataFrame(all_tweets, columns=["Tweets" ])

In [ ]:
dados.head()

In [ ]:
dados.drop_duplicates(inplace = True)

In [ ]:
dados['Tweets'].value_counts(normalize = True)

##Analise de sentimentos dados recentes na home_timeline

In [ ]:
 tweets = api.home_timeline(count=200)

In [ ]:
class TweetAnalyzer():
    
    def clean_tweet(self, tweet):
          return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))

        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):
        
        dados = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        
        dados['len'] = np.array([len(tweet.text) for tweet in tweets])
        dados['date'] = np.array([tweet.created_at for tweet in tweets])
        dados['source'] = np.array([tweet.source for tweet in tweets])
        dados['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        dados['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        dados['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in dados['tweets']])
        
        return dados

In [ ]:
tweet_analyzer = TweetAnalyzer()

In [ ]:
dados = tweet_analyzer.tweets_to_data_frame(tweets)

##Análise de sentimentos

### 0 - sentimento negativo
### 1 - sentimento positivo
#### -1 - sentimento neutro


In [ ]:
dados.head()

In [ ]:
sns.set()
ax = dados['sentiment'].value_counts().plot.bar()
ax.set_ylabel('Quantidade')
ax.set_xlabel('Sentiment')

In [ ]:
dados = tweet_analyzer.tweets_to_data_frame(tweets)
dados['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in dados['tweets']])

In [ ]:
print(dados.head(10))

In [ ]:
import seaborn as sns
import matplotlib as plt

In [ ]:
sns.set()
ax = dados['retweets'].value_counts().plot.bar()
ax.set_title('Total de retweets ')
ax.set_ylabel('Quantidade')
ax.set_xlabel('Retweets')

###LIKES POR DATA


In [ ]:
sns.set()
time_likes = pd.Series(data=dados['len'].values, index=dados['date'])
time_likes.plot(figsize=(16, 4), color='b')


#### RETWEETS POR DATA

In [ ]:
time_retweets = pd.Series(data=dados['retweets'].values, index=dados['date'])
time_retweets.plot(figsize=(16, 4), color='b')